## MF 796: Computational Methods of Mathematical Finance
Professors Christopher Kelliher and Eugene Sorets Spring 2024

### Problem set  5

#### Problem 1: Numerical PDEs:
Suppose that the underlying security SPY evolves according to standard geometric brownian motion. Then its derivatives obey the Black-Scholes equation:  
$$
\frac{\partial c}{\partial t} + \frac{1}{2}\sigma^2 s^2 \frac{\partial^2 c}{\partial s^2} + rs \frac{\partial c}{\partial s} - rc = 0 \quad (1)
$$
Assume SPY closed at 514 on March 14, 2024 and that the risk-free rate was 5.0%.  
We are going to find the price of a call spread with the right of early exercise. The two strikes of the call spread are $K_1 = 525$ and $K_2 = 540$ and the expiry is September 20, 2024.

##### (a) Explain why this instrument is not the same as being long an American call with strike 525 and short an American call with strike 540, both with expiry September 20, 2024

The difference lies in the early exercise rights.

When someone holds a long position in an American call with strike 525 and a short position in an American call with strike 540, each option can be exercised independently by the respective option holders. For example, this means that the short position can be assigned at any time, regardless of exercise the long call or not.  

On the other hand, a call spread with the right of early exercise is a single instrument. The holder have the right to exercise the spread as a whole.  

#####  (b) Let’s assume that we are not able to find σ by calibrating to the European call spread price and must find it by other means. Find a way to pick the σ, explain why you chose this method, and then find the σ

Use historical volatility as an estimate

In [100]:
import numpy as np
import pandas as pd
import yfinance as yf

symbol = "SPY"
start_date = "2023-11-15"
end_date = "2024-03-14"

stock_data = yf.download(symbol, start=start_date, end=end_date)

log_returns = np.log(stock_data["Close"] / stock_data["Close"].shift(1))

daily_std = log_returns.std()

historical_volatility = daily_std * np.sqrt(252)

print(f"historical volatility of SPY: {historical_volatility:.4f}")

[*********************100%%**********************]  1 of 1 completed

historical volatility of SPY: 0.1037


##### (c) Set up an explicit Euler discretization of (1). You will need to make decisions about the choice of $s_{max}$, $h_s$, $h_t$ , etc. Please explain how you arrived at each of these choices.

To set up an explicit Euler discretization of the Black-Scholes equation (1), we need to make decisions about the following parameters:

1. $s_{max}$: The maximum stock price in the grid.
2. $h_s$: The stock price step size.
3. $h_t$: The time step size.
   
Below is the numerical value of these parameters, and the reason why we choose this number.

1. $s_{max}$: We want to choose an $s_{max}$ that is sufficiently large to cover the range of stock prices we are interested in. For this question, since the highest strike price is 540, we can choose $s_{max}$ = 1620.

2. $h_s$: The stock price step size should be small enough to provide accurate results but not so small that it significantly increases the computational time. Let's choose $h_s$ = 5.

3. $h_t$: The time step size should be chosen to ensure stability and accuracy of the numerical scheme. For the explicit Euler method, the stability condition is  
$$ 
h_t \leq \frac{h_s^2}{\sigma^2s_{max}^2} 
$$

With $h_s$ = 1 and σ = 0.1037, we get $h_t \leq 0.00003$.

In [73]:
S0 = 514.0
K1 = 525.0
K2 = 540.0
sigma = 0.1037  # Volatility
smax = 3 * K2
r = 0.05  # Annual risk-free rate
T = 0.5  # 6 months
hs = 5.0  # Spatial step size
ht = 0.0001  # Time step size, adjusted for stability and accuracy

In [88]:
S = np.arange(hs, smax + hs, hs)
si = S[:-1]  # Exclude the last point to match the intended size
M = len(si)  # Correctly adjusting M to match the size of si

# Coefficients for the tridiagonal matrix
ai = 1 - (sigma**2) * (si**2) * (ht/hs**2) - r * ht
li = ((sigma**2) * (si**2)/2) * (ht/hs**2) - (r * si * ht)/(2*hs)
ui = ((sigma**2) * (si**2)/2) * (ht/hs**2) + (r * si * ht)/(2*hs)

# Initialize the tridiagonal matrix A
A = np.diag(ai)

# Adjustments for the tridiagonal matrix
l = li[1:]
u = ui[:-1]
for i in range(M-1):  # Adjusting the range to match the dimensions of A
    if i < M-2:  # Ensure we don't exceed the bounds
        A[i, i+1] = u[i]
    if i > 0: 
        A[i, i-1] = l[i-1]
A

array([[ 9.99993925e-01,  3.03768450e-06,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.84926200e-06,  9.99990699e-01,  7.15073800e-06, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.66083950e-06,  9.99985322e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         8.89187903e-01,  5.62060486e-02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.49442797e-02,  8.88496441e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  8.87802828e-01]])

##### (d) Let A be the update matrix that you created in the previous step. Find out its eigenvalues and check their absolute values. If necessary, make adjustments to $h_s$, $h_t$ , and anything else you need.

In [67]:
egv,egl = np.linalg.eig(A)

In [68]:
egl

array([[ 5.13877331e-13,  4.95169662e-13,  3.81468765e-13, ...,
         5.05007509e-23, -5.29210335e-23,  0.00000000e+00],
       [-1.07672573e-12, -3.54359409e-12, -1.05359657e-11, ...,
         4.75297028e-22, -4.88366217e-22,  0.00000000e+00],
       [ 7.52587670e-11,  2.12235575e-10,  5.89493906e-10, ...,
        -4.52987840e-22,  4.76349375e-22,  0.00000000e+00],
       ...,
       [-2.48439834e-07,  2.17212249e-07, -1.89349393e-07, ...,
        -2.02474926e-01,  1.79610148e-01,  0.00000000e+00],
       [-1.22928846e-07,  1.07452867e-07, -9.36488256e-08, ...,
         1.27026690e-01, -1.20263039e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [69]:
egv

array([0.99953911, 0.9995644 , 0.99958877, 0.99963482, 0.99965654,
       0.99969742, 0.99973501, 0.99975261, 0.99976943, 0.99978549,
       0.99980079, 0.99981537, 0.99982922, 0.99984237, 0.99985484,
       0.99986663, 0.99987776, 0.99988825, 0.99989811, 0.99999108,
       0.99998823, 0.99998645, 0.99998571, 0.99998514, 0.99998412,
       0.99998268, 0.99998084, 0.9999786 , 0.99997596, 0.99997289,
       0.9999694 , 0.99996545, 0.99996105, 0.99995616, 0.99995079,
       0.99994491, 0.9999385 , 0.99990735, 0.99993156, 0.99992406,
       0.999916  , 0.99930203, 0.9993352 , 0.99936732, 0.99939842,
       0.99945762, 0.99948574, 0.9995129 , 0.99961224, 0.99971662,
       0.9996774 , 0.99895405, 0.99899731, 0.99926781, 0.99923251,
       0.99903938, 0.99919613, 0.99908028, 0.99912003, 0.99915864,
       0.99942852, 0.99802023, 0.99815096, 0.99821411, 0.99833607,
       0.99839493, 0.99845238, 0.99850846, 0.9988639 , 0.99856318,
       0.99861655, 0.99881697, 0.9986686 , 0.99871934, 0.99876

In [71]:
check = np.any(np.abs(egv) > 1)
check

False

All the eigen value is less than 1, which means the stability condition is met

##### (e) Apply your discretization scheme to find today’s price of the call spread without the right of early exercise. The scheme will produce a whole vector of prices at time 0. Explain how you chose the one for today’s price.

In [96]:
def European_call_spread_price(smax, sigma, r, K1, K2, T, S0, hs, ht, M, N):
    # Grid setup for the underlying asset
    S = np.arange(0, smax + hs, hs)
    si = S[:-1]

    # Coefficients for the finite difference scheme
    ai = 1 - (sigma**2) * (si**2) * (ht/hs**2) - r * ht
    li = ((sigma**2) * (si**2) / 2) * (ht/hs**2) - (r * si * ht / (2 * hs))
    ui = ((sigma**2) * (si**2) / 2) * (ht/hs**2) + (r * si * ht / (2 * hs))

    # Construction of the tridiagonal matrix
    A = np.diag(ai, 0) + np.diag(li[1:], -1) + np.diag(ui[:-1], 1)
    C = np.zeros([M+1 , N])
    pl = np.maximum(si - K1, 0)  # Payoff for a call option with strike K1
    ps = np.maximum(si - K2, 0)  # Payoff for a call option with strike K2
    C[:, -1] = pl - ps  # Initial condition: max payoff at maturity

    for j in range(N, 1, -1):
        tj = ht * j
        bj = ui[-1] * (K2 - K1) * np.exp(-r * (T - tj))  # Boundary condition
        C[:, j - 2] = A @ C[:, j - 1]  # Propagate the solution backwards
        C[-1, j - 2] += bj  # Adjust the last spatial node with the boundary condition

    return np.interp(S0, si, C[:, 0])

S0 = 514.0
K1 = 525.0
K2 = 540.0
T = 0.5
r = 0.05
sigma = 0.1037
smax = 3 * K2
M = 275
N = 1000

hs = smax/M
ht = T/N

p1 = European_call_spread_price(smax, sigma, r, K1, K2, T, S0, hs, ht, M, N)
print(p1)


6.261705883665412


##### (f) Modify your code in the previous step to calculate the price of the call spread with the right of early exercise. What is the price?

In [97]:
def American_call_spread_price(smax, sigma, r, K1, K2, T, S0, hs, ht, M, N):
    # Grid setup for the underlying asset
    S = np.arange(0, smax + hs, hs)
    si = S[:-1]

    # Coefficients for the finite difference scheme
    ai = 1 - (sigma**2) * (si**2) * (ht/hs**2) - r * ht
    li = ((sigma**2) * (si**2) / 2) * (ht/hs**2) - (r * si * ht / (2 * hs))
    ui = ((sigma**2) * (si**2) / 2) * (ht/hs**2) + (r * si * ht / (2 * hs))

    # Construction of the tridiagonal matrix
    A = np.diag(ai, 0) + np.diag(li[1:], -1) + np.diag(ui[:-1], 1)
    C = np.zeros([M+1 , N])
    pl = np.maximum(si - K1, 0)  # Payoff for a call option with strike K1
    ps = np.maximum(si - K2, 0)  # Payoff for a call option with strike K2
    C[:, -1] = pl - ps  # Initial condition: max payoff at maturity

    for j in range(N, 1, -1):
        tj = ht * j
        bj = ui[-1] * (K2 - K1) * np.exp(-r * (T - tj))  # Boundary condition
        C[:, j - 2] = A @ C[:, j - 1]  # Propagate the solution backwards
        C[-1, j - 2] += bj  # Adjust the last spatial node with the boundary condition
        C[:,j-2] = np.max([C[:,j-2], pl-ps], axis=0)

    return np.interp(S0, si, C[:, 0])

p2 = American_call_spread_price(smax, sigma, r, K1, K2, T, S0, hs, ht, M, N)
print(p2)

8.75836431848145


##### (g) Calculate the early exercise premium as the difference between the American and European call spreads. Is it reasonable?

In [99]:
diff = p2-p1
diff

2.4966584348160374

The early exercise premium is about: 2.50. I think this result is reasonable. American call spreads allow people early exercise, so it is more expensive.